# Features Data

In [28]:
import json
import ipywidgets as widgets
import pandas as pd 
from IPython.core.display import display, HTML, clear_output
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Dropdown, IntText, Text
import requests
from os.path import exists

In [29]:
f = open('response.json',)
response = json.load(f)
rarities = response['rarities']
features = response['features']
total_mints = 1000
test_url = "https://lh3.googleusercontent.com/jCYKNAQFDdfRStZyqf57SZWmlfsXWaqL-p1-k4dlmxDwsab60j5XEv-p97WKWowDSp_w0OcUgfhvdCY6ZUWFuZJVPuOf8FFtpjg_uQ"

### Buy Feature 

In [30]:
feature_keys = list(response['features'].keys())
feature_menu = widgets.Dropdown(
       options=feature_keys,
       description='Features:')
feature_price = FloatText(
    description='Max Price:'
)
feature_button = widgets.Button(
        description='Buy',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Buy',
        icon='dollar'
    )
feature_out = widgets.Output()
def feature_on_button_clicked(_):
    # "linking function with output"
    with feature_out:
        # what happens when we press the button
        clear_output()
        option = feature_menu.value
        price = feature_price.value
        print('Buying {} at max price {} eth'.format(option,price))
    
feature_button.on_click(feature_on_button_clicked)              
feature_box = widgets.HBox([feature_menu,feature_price,feature_button,feature_out])
feature_box




### Buy Mint

In [31]:
mint_number = IntText(
    description='Mint Number:'
)
mint_price = FloatText(
    description='Max Price:'
)
mint_button = widgets.Button(
        description='Buy',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Buy',
        icon='dollar'
    )
mint_out = widgets.Output()
def mint_on_button_clicked(_):
    # "linking function with output"
    with mint_out:
        # what happens when we press the button
        clear_output()
        mint = mint_number.value
        price = mint_price.value
        print('Buying mint {} at max price {} eth'.format(mint,price))
    
mint_button.on_click(mint_on_button_clicked)              
mint_box = widgets.HBox([mint_number,mint_price,mint_button,mint_out])
mint_box

In [32]:
outputs = [widgets.Output() for i in range(len(feature_keys))]
tab = widgets.Tab(children = outputs)

for i in range(len(feature_keys)):
    tab.set_title(i, feature_keys[i])

for i in range(len(outputs)): 
    with outputs[i]: 
        display(HTML('<h1>{}</h1>'.format(feature_keys[i])))
        data = features[feature_keys[i]]
        for d in data: 
            name = d['name']
            mints_count = len(d['mints'])
            display(HTML('<h2>{}</h2>({} mints with this feature - {}% of collection)'.format(name,mints_count,mints_count/total_mints * 100 )))        
            feature_outputs = [widgets.Output() for i in range(2)]
            feature_tab = widgets.Tab(children = feature_outputs)
            feature_tab.set_title(0, "data")
            feature_tab.set_title(1, "gallery")
            with feature_outputs[0]:
                mints = d['mints'][0:20]
                mints_df = pd.DataFrame.from_dict(mints)
                mints_df = mints_df[['rank', 'mint_number', 'price', 'os_link', 'render_link']]
                display(HTML(mints_df.to_html(escape=False)))
            with feature_outputs[1]:
                mints = d['mints'][0:20]
                images = []
                mints_df = pd.DataFrame.from_dict(mints)
                for index,row in mints_df.iterrows(): 
                    true_mint_num = row['mint_number']
                    mint_num = "12"
                    render_link = row['render_link']
                    file_exists = exists(mint_num) 
                    if not file_exists: 
                        response = requests.get(test_url)
                        f = open(mint_num, "wb")
                        f.write(response.content)
                        f.close()   
                    n = open(mint_num, "rb")
                    byt = n.read()
                    img = widgets.Image(
                        value=byt,
                        format='png',
                        width=300,
                        height=400,
                        description="render"
                    )
                    text = Text(value=str(true_mint_num));
                    box = widgets.VBox([text,img])
                    images.append(box)
                img_box = widgets.HBox(images)
                display(img_box)
            display(feature_tab)


tab